<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/41_water_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [1]:
# !pip install geemap

# Mapping surface water and mesic resources dynamics with the Sentinel Fusion product

**Steps to create Sentinel Fusion timeseries:**

1. Pan and zoom to your area of interest (AOI), and click on the map to select a polygon. Alternatively, you can enable `Use user-drawn AOI` and use the Drawing tools (e.g., rectangle) to draw a shape on the map.
 
2. Click the `Submit` button to create timeseries of water resources in a given HUC12 unit within the High Divide or a user drawn AOI.

3. Adapted from code developed by [Dr. Qiusheng Wu](https://wetlands.io/)

**Contact:** Nick Kolarik ([Website](https://www.boisestate.edu/hes/people/nick-kolarik/#profile), [Twitter](https://twitter.com/nickkolarik), [Google Scholar](https://scholar.google.com/citations?user=ByYTxW8AAAAJ&hl=en&oi=ao))

In [1]:
# Check geemap installation
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [2]:
# Import libraries
import os
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

In [3]:
# Create an interactive map
Map = geemap.Map(center=[45, -113.5], zoom=7, add_google_map=False)
Map.add_basemap('HYBRID')
Map.add_basemap('ROADMAP')

# Add Earth Engine data
hd = ee.FeatureCollection("users/nekolarik/HD_boundary")
fc = ee.FeatureCollection("USGS/WBD/2017/HUC12").filterBounds(hd)
Map.addLayer(fc, {}, 'HUC12 High Divide')

#states = ee.FeatureCollection('TIGER/2018/States')
# Map.addLayer(states, {}, 'US States')

Map

Map(center=[45, -113.5], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(valu…

In [4]:
# Design interactive widgets

style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)


admin_widget = widgets.Text(
    description='HUC12 Name:', value='', width=300, style=style
)

aoi_widget = widgets.Checkbox(
    value=False, description='Use user-drawn AOI', style=style
)

download_widget = widgets.Checkbox(
    value=False, description='Download chart data', style=style
)


def aoi_change(change):
    Map.layers = Map.layers[:4]
    Map.user_roi = None
    Map.user_rois = None
    Map.draw_count = 0
    admin_widget.value = ''
    output_widget.clear_output()


aoi_widget.observe(aoi_change, names='value')

resource = widgets.Dropdown(
    description='Land cover:',
    options=[
        'Surface Water',
        'Mesic',
        'Other' ],
    value='Surface Water',
    style=style,
)

month = widgets.Dropdown(
    description='Month:',
    options=[
        'June',
        'July',
        'August',
        'September'],
    value='July',
    style=style,
)
#year_widget = widgets.IntSlider(
#    min=1984, max=2020, value=2010, description='Selected year:', width=400, style=style
#)


#fmask_widget = widgets.Checkbox(
#    value=True, description='Apply fmask(remove cloud, shadow, snow)', style=style
#)



color = widgets.ColorPicker(
    concise=False, description='Color:', value='blue', style=style
)



submit = widgets.Button(
    description='Submit', button_style='primary', tooltip='Click me', style=style
)

full_widget = widgets.VBox(
    [
        widgets.HBox([admin_widget, aoi_widget, download_widget]),
        #widgets.HBox([band_combo, year_widget, fmask_widget]),
        widgets.HBox([resource, month, color]),
        #widgets.HBox([color]),
        submit,
    ]
)

full_widget

In [5]:
# Capture user interaction with the map


def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click' and not aoi_widget.value:
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        selected_fc = fc.filterBounds(xy)

        with output_widget:
            output_widget.clear_output()

            try:
                feature = selected_fc.first()
                admin_id = feature.get('name').getInfo()
                #statefp = feature.get('states')

                admin_widget.value = admin_id
                Map.layers = Map.layers[:4]
                geom = selected_fc.geometry()
                layer_name = admin_id 
                Map.addLayer(
                    ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name
                )
                print(layer_name)
            except:
                print('No feature could be found')
                Map.layers = Map.layers[:4]

        Map.default_style = {'cursor': 'pointer'}
    else:
        Map.draw_count = 0


Map.on_interaction(handle_interaction)

In [6]:
# Click event handler

images = ee.ImageCollection(['users/nekolarik/SF_MS/juneSF', 'users/nekolarik/SF_MS/julySF',
                           'users/nekolarik/SF_MS/augSF', 'users/nekolarik/SF_MS/sepSF'])

def submit_clicked(b):

    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:
            admin_id = admin_widget.value
            #admin2_id = admin2_widget.value
            #band1 = first_band.value
            #band2 = second_band.value
            #selected_year = year_widget.value
            #threshold = nd_threshold.value
            #bands = band_combo.value.split('/')
            #apply_fmask = fmask_widget.value
            #selected_month = month.value
            palette = color.value
            use_aoi = aoi_widget.value
            download = download_widget.value

            if use_aoi:
                if Map.user_roi is not None:
                    roi = Map.user_roi
                    layer_name = 'User drawn AOI'
                    geom = roi
                else:
                    output_widget.clear_output()
                    print('No user AOI could be found.')
                    return
            else:

                #statefp = ee.Feature(
                    #fc.filter(ee.Filter.eq('name', admin1_id)).first()
                #).get('states')
                roi = fc.filter(
                    ee.Filter.And(
                        ee.Filter.eq('name', admin_id),
                        #ee.Filter.eq('states', statefp),
                    )
                )
                layer_name = admin_id
                geom = roi.geometry()

            Map.layers = Map.layers[:4]
            Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)

           
            #images = SFimgs(
                #roi=roi
                #start_year=1984,
                #end_year=2020,
                #start_date='01-01',
                #end_date='12-31',
                #apply_fmask=apply_fmask,
            #)
            #nd_images = images.map(lambda img: img.normalizedDifference([band1, band2]))
            #result_images = nd_images.map(lambda img: img.gt(threshold))
            
            ## coding the strings to class vals
            if resource.value == 'Surface Water':
                res_val = 3
            elif resource.value == 'Mesic':
                res_val = 2
            else:
                res_val = 1
            
            ## assigning image indexes to month string vals
            if month.value == 'June':
                selected_month = 0
            if month.value == 'July':
                selected_month = 1
            if month.value == 'August':
                selected_month = 2
            if month.value == 'September':
                selected_month = 3
                
            
            result_images = images.map(lambda img: img.eq(res_val))

            selected_image = ee.Image(
                images.toList(images.size()).get(selected_month)
            )
            selected_result_image = ee.Image(
                result_images.toList(result_images.size()).get(selected_month)
            ).selfMask().clip(roi)

            #vis_params = {'bands': bands, 'min': 0, 'max': 3000}

            #Map.addLayer(selected_image, vis_params, 'Landsat ' + str(selected_year))
            Map.addLayer(
                selected_result_image,
                {'palette': palette},
                'Result ' + month.value,
            )

            def cal_area(img):
                pixel_area = img.multiply(ee.Image.pixelArea()).divide(1e4)
                img_area = pixel_area.reduceRegion(
                    **{
                        'geometry': geom,
                        'reducer': ee.Reducer.sum(),
                        'scale': 10,
                        'maxPixels': 1e12,
                        'bestEffort': True,
                    }
                )
                return img.set({'area': img_area})

            areas = result_images.map(cal_area)
            stats = areas.aggregate_array('area').getInfo()
            print(stats)
            x = ['June', 'July', 'Aug', 'Sep']
            
            y = [item.get('classification') for item in stats]
            

            fig = plt.figure(1)
            fig.layout.height = '270px'
            plt.clear()
            plt.plot(x, y)
            plt.title('Temporal trend')
            plt.xlabel('Month')
            plt.ylabel('Area (ha)')

            output_widget.clear_output()

            plt.show()

            if download:
                out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                out_name = 'chart_' + geemap.random_string() + '.csv'
                out_csv = os.path.join(out_dir, out_name)
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)
                with open(out_csv, 'w') as f:
                    f.write('year, area (ha)\n')
                    for index, item in enumerate(x):
                        line = '{},{:.2f}\n'.format(item, y[index])
                        f.write(line)
                link = geemap.create_download_link(
                    out_csv, title="Click here to download the chart data: "
                )
                display(link)

        except Exception as e:
            print(e)
            print('An error occurred during computation.')

        Map.default_style = {'cursor': 'default'}


submit.on_click(submit_clicked)